In [11]:
import os
#import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
import numpy
import datetime
from pyspark import SparkContext
from pyspark import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql import GroupedData
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix, RowMatrix
from pyspark import RDD

In [2]:
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3.6'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/usr/bin/python3.6'
os.environ['SPARK_HOME'] = '/usr/bin/python3.6'

#data = pd.read_csv('data_engineered_features500.csv', nrows=60018)
#data = data.query("userid == ['user_000001','user_000002','user_000003','user_000004','user_000005']")

#weekend = 1 means it is a weekend
#data = data[['userid', 'track-name', 'artist-name', 'weekend', 'skipped']]

In [3]:
sc = SparkSession \
    .builder \
    .appName("Python Spark tSVDpp") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
    
sqlContext = SQLContext(sc)

In [4]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType, FloatType
from pyspark.ml.feature import VectorAssembler

schema = StructType([
    StructField("userid", StringType()), #init
    StructField("track-name", StringType()), #init
    StructField("artist-name", StringType()), #init    
    
    StructField("timestamp", StringType()),
    StructField("weekday", IntegerType()),
    StructField("hour", IntegerType()),
    
    StructField("weekend", FloatType()), #init
    
    StructField("daytime", IntegerType()),
    StructField("track-total-count", IntegerType()),
    StructField("track-weekday-daytime-count", IntegerType()),
    StructField("songlength", FloatType()),
    StructField("last-seen-song", FloatType()),
    StructField("month", IntegerType()),
    StructField("quarter", IntegerType()),
    
    StructField("skipped", FloatType()), #init
    
    StructField("artist-total-count", IntegerType()),
    StructField("artist-weekday-daytime-count", IntegerType()),
    StructField("last-seen-artist", FloatType()),
    StructField("age", IntegerType()),
    StructField("gender_int", IntegerType()),
])

In [5]:
df = sc.read.csv("data_engineered_features500.csv", header=True, schema=schema) \
    .select(['userid','track-name','artist-name','weekend','skipped'])
df.printSchema()

root
 |-- userid: string (nullable = true)
 |-- track-name: string (nullable = true)
 |-- artist-name: string (nullable = true)
 |-- weekend: float (nullable = true)
 |-- skipped: float (nullable = true)



In [6]:
#df.printSchema()
df = df.groupby(['userid','track-name','artist-name','weekend']) \
    .mean('skipped') \
    .sort('userid')

#df.show()

In [7]:
#df.rdd.map()
#df.registerAsTable('df_table')
#sqlContext.sql("""select userid, track-name, artist-name, weekend,
#                avg(skipped) as skipped
#                from df_table
#                groupby userid, track-name, artist-name, weekend""").show()

In [8]:
rows = sc.sparkContext.parallelize(df.rdd.collect())
#mat = sc.IndexedRowMatrix(df)

In [12]:
mat = RowMatrix(rows)

In [25]:
mat.numRows()

Py4JJavaError: An error occurred while calling o129.numRows.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 12.0 failed 1 times, most recent failure: Lost task 2.0 in stage 12.0 (TID 625, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 123, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 2.7 than that in driver 3.6, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.apache.spark.api.python.PythonRunner$$anon$1.<init>(PythonRDD.scala:234)
	at org.apache.spark.api.python.PythonRunner.compute(PythonRDD.scala:152)
	at o

In [18]:
svd_model = mat.computeSVD(1, computeU=True)

Py4JJavaError: An error occurred while calling o129.computeSVD.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 614, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 123, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 2.7 than that in driver 3.6, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.apache.spark.api.python.PythonRunner$$anon$1.<init>(PythonRDD.scala:234)
	at org.apache.spark.api.python.PythonRunner.compute(PythonRDD.scala:152)
	a